In [1]:
import os
import re
from config import cfg

result_dir = 'search-results'
cfg_dir = 'configs/search'
best_name = ''
best_macro = 0
best_macro_std = 0
correspond_micro = 0
correspond_micro_std = 0
cnt = 0

best_selective_macro = 0
best_selective_name = 0
for file_name in os.listdir(result_dir):
    config_path = os.path.join(cfg_dir, file_name[:-4])
    cfg.merge_from_file(config_path)
    file_path = os.path.join(result_dir, file_name)
    with open(file_path, 'r') as f:
        lines = f.readlines()
        if len(lines) < 2:
            continue
        line_macro = lines[-2]
        line_micro = lines[-1]
        cnt += 1
        if line_macro.find('Macro') == -1 or line_micro.find('Micro') == -1:
            continue

        macro_mean, macro_std = float(line_macro[16:22]), float(line_macro[26:32])
        micro_mean, micro_std = float(line_micro[16:22]), float(line_micro[26:32])
        # if cfg.model.selective == True:
        #     if best_selective_macro < macro_mean:
        #         best_name = file_name
        #         best_macro = macro_mean
        #         best_macro_std = macro_std
        #         correspond_micro = micro_mean
        #         correspond_micro_std = micro_std
        
        if best_macro < macro_mean:
            best_name = file_name
            best_macro = macro_mean
            best_macro_std = macro_std
            correspond_micro = micro_mean
            correspond_micro_std = micro_std

print(f'Best model in {cnt} estimated configs: {best_name}')
print(f'Macro F1: {best_macro} +- {best_macro_std}')
print(f'Micro F1: {correspond_micro} +- {correspond_micro_std}')
        


Best model in 9 estimated configs: fb-mit-8.yaml.txt
Macro F1: 73.33 +- 2.2
Micro F1: 76.3 +- 1.74


In [15]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats


def calculate_mean_sem(data, axis=0):
    """Calculate the mean and standard error of the mean for each epoch and variable."""
    means = np.mean(data, axis=axis)
    sems = np.array([scipy.stats.sem(data[:, i]) for i in range(data.shape[1])])
    return means, sems

def calculate_confidence_interval(mean, sem, confidence_level=1.96):
    """Calculate the 95% confidence interval."""
    ci = sem * confidence_level
    lower_bound = mean - ci
    upper_bound = mean + ci
    return lower_bound, upper_bound

#

In [18]:
gammas = np.load('plots/raw/actor_gammas.npy', allow_pickle=True).item()
colors = ['#f0c38e', '#6E85B2', '#f38181']

for key, data in gammas.items():
    lambda_names = key.split('-')
    # Calculate means and SEMs
    means, sems = calculate_mean_sem(data, axis=0)

    # Calculate confidence intervals
    lower_bounds, upper_bounds = calculate_confidence_interval(means, sems)

    epochs = np.arange(means.shape[0])
    plt.clf()
    plt.figure(figsize=(8, 3))
    for i in range(means.shape[1]):
        plt.plot(epochs, means[:, i], label=f'{lambda_names[i]}_{i}', color=colors[i])
        plt.fill_between(epochs, lower_bounds[:, i], upper_bounds[:, i], alpha=0.15, color=colors[i])

        plt.title(f'{key}')
        plt.legend()
    plt.tight_layout()
    plt.savefig(f'./plots/actor/{key}.png', dpi=300)
    plt.close()

<Figure size 640x480 with 0 Axes>

In [14]:
# For simple attention gates 

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

head_gates = np.load('plots/raw/actor_head_gates_simple.npy', allow_pickle=True)

means, sems = calculate_mean_sem(head_gates, axis=0)
lower_bounds, upper_bounds = calculate_confidence_interval(means, sems, confidence_level=0.3)
epochs = np.arange(means.shape[0])

clip = 50
plt.clf()
plt.figure(figsize=(8, 3))

# '#c7ffd8', 
colors = ['#95e1d3', '#f0c38e', '#6E85B2', '#f38181']

for i in range(means.shape[1]):
    plt.plot(epochs[:clip], means[:clip, i], label=f'Gate {i}', color=colors[i])
    plt.fill_between(epochs[:clip], lower_bounds[:clip, i], upper_bounds[:clip, i], alpha=0.15, color=colors[i])
    # break
plt.legend()
plt.tight_layout()
# plt.show()
plt.savefig(f'./plots/actor/attention_gate_simple.png', dpi=300)
plt.close()


<Figure size 640x480 with 0 Axes>

In [13]:
# For linear attention gates 

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

head_gates = np.load('plots/raw/actor_head_gates.npy', allow_pickle=True)

tgt_node = 0
tgt_token = 0

data = head_gates[:, :, tgt_node, tgt_token, :]

means, sems = calculate_mean_sem(data, axis=0)
lower_bounds, upper_bounds = calculate_confidence_interval(means, sems, confidence_level=0.3)

print(data.shape)


epochs = np.arange(100)

clip = 50
plt.clf()
plt.figure(figsize=(8, 3))
colors = ['#95e1d3', '#f0c38e', '#6E85B2', '#f38181']
for i in range(data.shape[-1]):
    plt.plot(epochs[:clip], means[:clip, i], label=f'Gate {i}', color=colors[i])
    plt.fill_between(epochs[:clip], lower_bounds[:clip, i], upper_bounds[:clip, i], alpha=0.15, color=colors[i])
    # break
plt.legend()
plt.tight_layout()
# plt.show()
plt.savefig(f'./plots/actor/attention_gate_linear.png', dpi=300)
plt.close()


(3, 100, 4)


<Figure size 640x480 with 0 Axes>